In [6]:
sfrom keras.datasets import mnist
import numpy as np
(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print (x_train.shape)
print (x_test.shape)

SyntaxError: ignored

In [0]:
from typing import Callable, List, Tuple, Union

from keras.layers import Dense, Input, Layer
from keras import Model


def _architecture_builder(input_shape: Tuple, output_shape: int, neurons: List[int]) -> List[Layer]:
    layers = [Input(input_shape)]
    for n in neurons:
        layers.append(Dense(n, activation='relu'))
    layers.append(Dense(output_shape, activation='sigmoid'))
    return layers


def _model_builder(layers: List[Layer]) -> Model:
    inputs = layers[0]
    layer = layers[1](inputs)
    for l in layers[2:]:
        layer = l(layer)
    model = Model(inputs, layer)
    return model


def dense_encoder_decoder(input_shape: Tuple, output_shape: int, encoder_size: int, neurons: List[int],
                          optimizer: Union[Callable, str] = 'adam', loss: Union[Callable, str] = 'mse')\
        -> Tuple[Model, Model, Model]:

    encoder_layers = _architecture_builder(input_shape, encoder_size, neurons)
    decoder_layers = _architecture_builder((encoder_size,), output_shape, neurons[::-1])
    encoder_decoder_layers = encoder_layers + decoder_layers[1:]
    encoder = _model_builder(encoder_layers)
    decoder = _model_builder(decoder_layers)
    encoder_decoder = _model_builder(encoder_decoder_layers)
    encoder_decoder.compile(optimizer=optimizer, loss=loss)
    return encoder_decoder, encoder, decoder

In [0]:
autoencoder, encoder, decoder = dense_encoder_decoder((784,), 784, 2, [784,128,64,32])

autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [9]:
autoencoder.fit(x_train, x_train,
                epochs=10,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

NameError: ignored

In [0]:
encoded_imgs = encoder.predict(x_test)
predicted = autoencoder.predict(x_test)

import random
x = np.full((10, 2), random.random())
generated = decoder.predict(x)